
# Clustering

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.feature_extraction.text
import sklearn.cluster
import sklearn.decomposition
#from adjustText import adjust_text
import altair as alt

import sys
sys.path.insert(0, '../../scripts')
import util
import theme_desarroio

import matplotlib.pyplot as plt

In [3]:
theme=theme_desarroio.theme_desarroio()

Loading file:

In [4]:
#file = '../../data/clean/speech.pkl'
file = '../../data/clean/speech_init.pkl'

In [5]:
df = pd.read_pickle(file)

In [6]:
df

,filename,raw text,year,cleaned text,administration,president,year-president,tokenized_words,normalized_words,tokenized_paragraphs,tokenized_words_in_paragraphs,normalized_words_in_paragraphs
0,mensaje-1963-fbt.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1963,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Belaunde(1),Belaunde,1963-Belaunde,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, ar...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, a..."
1,mensaje-1969-jva.txt,"[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\...",1969,"MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\n...",Velasco,Velasco,1969-Velasco,"[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL,...","[mensaje, nación, presidente, perú, división, ...","[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, ...","[[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL...","[[mensaje, nación, presidente, perú, división,..."
2,mensaje-1976-fmb.txt,"[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\...",1976,"MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\n...",Morales Bermudez,Morales Bermudez,1976-Morales Bermudez,"[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL,...","[mensaje, nación, presidente, perú, división, ...","[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, ...","[[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL...","[[mensaje, nación, presidente, perú, división,..."
3,mensaje-1980-fbt.txt,"[ARQUITECTO FERNANDO BELAUNDE TERRY, \n, \n, A...",1980,"ARQUITECTO FERNANDO BELAUNDE TERRY, \n\nANTE E...",Belaunde(2),Belaunde,1980-Belaunde,"[ARQUITECTO, FERNANDO, BELAUNDE, TERRY, ANTE, ...","[arquitecto, fernando, belaunde, terry, congre...","[ARQUITECTO FERNANDO BELAUNDE TERRY, ANTE EL C...","[[ARQUITECTO, FERNANDO, BELAUNDE, TERRY, ANTE,...","[[arquitecto, fernando, belaunde, terry, congr..."
4,mensaje-1985-ag.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1985,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Garcia(1),Garcia,1985-Garcia,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, do...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, d..."
5,mensaje-1990-af.txt,"[INGENIERO ALBERTO FUJIMORI FUJIMORI, \n, \n, ...",1990,"INGENIERO ALBERTO FUJIMORI FUJIMORI, \n\nANTE ...",Fujimori(1),Fujimori,1990-Fujimori,"[INGENIERO, ALBERTO, FUJIMORI, FUJIMORI, ANTE,...","[ingeniero, alberto, fujimori, fujimori, congr...","[INGENIERO ALBERTO FUJIMORI FUJIMORI, ANTE EL ...","[[INGENIERO, ALBERTO, FUJIMORI, FUJIMORI, ANTE...","[[ingeniero, alberto, fujimori, fujimori, cong..."
6,mensaje-1995-af.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1995,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Fujimori(2),Fujimori,1995-Fujimori,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, in...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, i..."
7,mensaje-2000-af.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,2000,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Fujimori(3),Fujimori,2000-Fujimori,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, in...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, i..."
8,mensaje-2000-vp-noviembre.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,2000,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Paniagua,Paniagua,2000-Paniagua,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, do...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucion

## Analysis description

The main purpose of this analysis is to explore the associations and patterns present in the word frequency of each speech. Specifically, we would like to assess if speeches within the same cluster signal similarities in political discourses or closeness in the left-right political spectrum.

To visualize the result, we will generate the first two components of the Principal Component Analysis (PCA) of the tf-idf matrix for each speech. Then we will do a scatter plot of each speech and its position according to these two principal components to see how are the clusters distributed.

## K-means

In [7]:
tf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=5, norm='l2')

In [8]:
df['normalized_words_concat'] = df['normalized_words'].apply(lambda x: ' '.join(x))

In [9]:
#col = 'cleaned text'
col = 'normalized_words_concat'

In [10]:
tf_vects = tf_vectorizer.fit_transform(df[col])

In [11]:
terms = tf_vectorizer.get_feature_names()

In [12]:
#n_clusters = [3, 4, 5, 6, 7, 8, 9, 10]
n_clusters = [2,3,4,5]

In [13]:
for n in n_clusters:
    km = sklearn.cluster.KMeans(n_clusters = n,
                                init = 'k-means++',
                                random_state = 995300)
    km.fit(tf_vects)
    df['kmeans' + str(n)] = km.labels_

## PCA

In [14]:
pca = sklearn.decomposition.PCA(n_components = 2, random_state = 254971).fit(tf_vects.toarray())
reduced_pca = pca.transform(tf_vects.toarray())

In [15]:
pca_df = pd.DataFrame()
pca_df['president'] = df['president']
pca_df['administration'] = df['administration']
pca_df['year'] = df['year']
pca_df['pca1'] = reduced_pca[:,0]
pca_df['pca2'] = reduced_pca[:,1]

for num in n_clusters:
    pca_df['clusters_'+str(num)] = df['kmeans'+str(num)]

#pca_df.loc[(pca_df['president']=='Fujimori') & (pca_df['year']=='2000'), 'administration'] = 'Fujimori(3)'

In [16]:
n_clusters = 3
pca_df['Cluster'] = pca_df['clusters_'+str(n_clusters)] + 1

In [17]:
pca_df

,president,administration,year,pca1,pca2,clusters_2,clusters_3,clusters_4,clusters_5,Cluster
0,Belaunde,Belaunde(1),1963,0.087063,-0.357548,0,1,0,4,2
1,Velasco,Velasco,1969,-0.205189,-0.265208,0,1,0,0,2
2,Morales Bermudez,Morales Bermudez,1976,-0.147707,-0.276087,0,1,0,0,2
3,Belaunde,Belaunde(2),1980,-0.051496,-0.215709,0,1,0,4,2
4,Garcia,Garcia(1),1985,-0.173658,-0.097349,0,1,0,0,2
5,Fujimori,Fujimori(1),1990,-0.191736,-0.115883,0,1,0,0,2
6,Fujimori,Fujimori(2),1995,-0.137610,0.175507,0,1,1,1,2
7,Fujimori,Fujimori(3),2000,-0.128187,0.107162,0,1,0,0,2
8,Paniagua,Paniagua,2000,0.077648,-0.279351,0,1,2,2,2
9,Toledo,Toledo,2001,-0.112955,0.302382,0,0,1,1,1


# Plot

In [18]:
labels_down = ['Morales Bermudez', 'Fujimori(1)', 'Fujimori(2)', 'Fujimori(3)']
labels_right = ['Garcia(2)']

pca_df_labels_down = pca_df[pca_df['administration'].isin(labels_down)]
pca_df_labels_right = pca_df[pca_df['administration'].isin(labels_right)]
pca_df_labels_up = pca_df[~((pca_df['administration'].isin(labels_down)) | (pca_df['administration'].isin(labels_right)))]

In [19]:
width = 700
height = 300

title = ["Análisis de clusters"]
subtitle = ['Visualización de componentes principales de la matriz TF-IDF de cada discurso',
            'Cada eje representa uno de los dos primeros componentes principales',
            'Pertenencia al mismo cluster y cercania en el grafico indican mayor similitud en las palabras incluidas en cada discurso',
            'Fuente: Congreso del Peru']

points_down = alt.Chart(pca_df_labels_down).mark_circle(
    size=60
    ).encode(
    x = alt.X('pca1:Q', axis=alt.Axis(labels=False), title=None),
    y = alt.Y('pca2:Q', axis=alt.Axis(labels=False), title=None),
    color = 'Cluster:N'
    ).properties(
    width = width,
    height = height,
    title={
        "text": title, 
        "subtitle": subtitle,
        "subtitleFont": "Sanchez"
    }
    )

texts_down = points_down.mark_text(
    align = 'center',
    baseline = 'middle',
    dy = 10
    ).encode(
    text = 'administration'
    )

points_up = alt.Chart(pca_df_labels_up).mark_circle(
    size=60
    ).encode(
    x = alt.X('pca1:Q', axis=alt.Axis(labels=False), title=None),
    y = alt.Y('pca2:Q', axis=alt.Axis(labels=False), title=None),
    color = 'Cluster:N'
    )

texts_up = points_up.mark_text(
    align = 'center',
    baseline = 'middle',
    dy = -10
    ).encode(
    text = 'administration'
    )

points_right = alt.Chart(pca_df_labels_right).mark_circle(
    size=60
    ).encode(
    x = alt.X('pca1:Q', axis=alt.Axis(labels=False), title=None),
    y = alt.Y('pca2:Q', axis=alt.Axis(labels=False), title=None),
    color = 'Cluster:N'
    )

texts_right = points_right.mark_text(
    align = 'center',
    baseline = 'middle',
    dx = 20,
    dy = -10
    ).encode(
    text = 'administration'
    )

chart = points_down + texts_down + points_up + texts_up + points_right + texts_right

chart = alt.concat(chart).properties(
    title = alt.TitleParams(
        ['Incluye el discurso inaugural de cada administración, incluyendo reelecciones continuas'],
        baseline = 'bottom',
        orient = 'bottom',
        #anchor = 'end',
        fontWeight = 'normal',
        fontSize = 11
    )
).configure_axis(
    grid = False,
    ticks = False
)

chart

alt.ConcatChart(...)

In [ ]:
output = '../../outputs/clusters.png'
chart.save(output)